# Analisador de empresas

Tem como objetivo extrair informações relacionadas as empresas de uma determinada região e sua classificação no Google Meu Negócio e Facebook, se houver mídia relacionada.

In [1]:
!pip install psycopg2-binary

In [2]:
DEBUG = True

def LOG(funcao, mensagem, pulaLinhaAntes = False, pulaLinhaDepois = False):
    if DEBUG == True:
        if pulaLinhaAntes:
            print('\n');
        print('[{}] => {}'.format(funcao, mensagem))
        if pulaLinhaDepois:
            print('\n');

In [13]:
from urllib.request import urlopen
import json
import psycopg2
import uuid

```sql
CREATE TABLE empresa (
    id           VARCHAR(36)   NOT NULL,
    nome         VARCHAR(255)  NOT NULL,
    atividade    VARCHAR(255)  NOT NULL,
    emailCliente VARCHAR(255),
    cep          VARCHAR(20) ,
    cidade       VARCHAR(255),
    estado       VARCHAR(255),
    bairro       VARCHAR(255),
    uf           VARCHAR(5)  ,
    PRIMARY KEY (id)
);

CREATE TABLE empresa_telefone (
    id                VARCHAR(36) NOT NULL,
    ddd               VARCHAR(5),
    telefone          VARCHAR(50) NOT NULL,
    telefoneCompleto  VARCHAR(100),
    whatsapp          BOOLEAN,
    telefone0800      BOOLEAN,
    PRIMARY KEY (id)
);

CREATE TABLE empresa_endereco (
    id       VARCHAR(36)  NOT NULL,
    endereco VARCHAR(255) NOT NULL,
    PRIMARY KEY (id)
);
```

In [32]:
def existeEmpresaComNome(cursor, nome):

    cursor.execute("SELECT 1 FROM empresa WHERE nome = '{}'".format(nome))

    if (cursor.fetchone() == None):
        return False
    else:
        return True

In [50]:
def inserirEmpresa(conexao, cursor, empresa):

    LOG('inserirEmpresaBancoDados', 'Iniciado')

    if (existeEmpresaComNome(cursor, empresa['nome']) == False):
        
        idEmpresa = uuid.uuid4()
        
        LOG('inserirEmpresaBancoDados', 'UUID do registro: {}'.format(str(idEmpresa)))
        
        cursor.execute("INSERT INTO empresa (id, nome, atividade, emailCliente, cep, cidade, estado, bairro, uf) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",(str(idEmpresa), empresa['nome'], empresa['atividade'], empresa['emailCliente'], empresa['cep'], empresa['cidade'], empresa['estado'], empresa['bairro'], empresa['uf']))
        conexao.commit()

        LOG('inserirEmpresaBancoDados', 'Registro inserido: {}'.format(empresa['nome']))

    LOG('inserirBancoDados', 'Finalizado')

    return

In [55]:
def inserirEmpresas(empresas):

    LOG('inserirEmpresas', 'Iniciado')

    nomeBancoDados = 'COLETADEDADOS'
    usuarioBancoDados = 'postgres'
    senhaBancoDados = '<senha>'
    hostBancoDados = 'localhost'

    conexao = psycopg2.connect(dbname=nomeBancoDados, user=usuarioBancoDados, password=senhaBancoDados, host=hostBancoDados)
    cursor = conexao.cursor()

    LOG('inserirEmpresas', 'Conexao com banco de dados: {} efetuada'.format(nomeBancoDados))

    for empresa in empresas:
        inserirEmpresa(conexao, cursor, empresa)

    cursor.close()
    conexao.close()

    LOG('inserirEmpresas', 'Conexao com banco de dados: {} finalizada'.format(nomeBancoDados))
    LOG('inserirEmpresas', 'Finalizado')

    return

In [52]:
def buscarEmpresasDaPagina(categoria, cidade, estado, pagina):

    LOG('buscarEmpresasDaPagina', 'Iniciado')

    site = 'https://guiafacil.com/buscaAjax.php?busca={}&cidade={}&tipoBusca=segmento&estado={}&pagina={}'
    site = site.format(categoria, cidade, estado, pagina)

    LOG('buscarEmpresasDaPagina', 'Buscar: {}'.format(site))

    conteudo = urlopen(site).read()
    conteudoJson = json.loads(conteudo)
    resultado = conteudoJson['resultados']

    LOG('buscarEmpresasDaPagina', 'Quantidade de empresas da página: {}'.format(len(resultado)))
    LOG('buscarEmpresasDaPagina', 'Finalizado')

    return resultado

In [53]:
def buscarEmpresas(categoria, cidade, estado):

    LOG('buscarEmpresas', 'Iniciado')

    resultado = []
    pagina = 1

    while(True):
        resultadoPagina = buscarEmpresasDaPagina(categoria, cidade, estado, pagina)
        if (resultadoPagina == 0):
            break

        inserirEmpresas(resultadoPagina)
        resultado += resultadoPagina
        pagina += 1
        break

    LOG('buscarEmpresas', 'Quantidade de empresas: {}'.format(len(resultado)))
    LOG('buscarEmpresas', 'Finalizado')

    return resultado

In [54]:
buscarEmpresas('', 'joinville', 'santa-catarina')

[buscarEmpresas] => Iniciado
[buscarEmpresasDaPagina] => Iniciado
[buscarEmpresasDaPagina] => Buscar: https://guiafacil.com/buscaAjax.php?busca=&cidade=joinville&tipoBusca=segmento&estado=santa-catarina&pagina=1
[buscarEmpresasDaPagina] => Quantidade de empresas da página: 20
[buscarEmpresasDaPagina] => Finalizado
[inserirEmpresas] => Iniciado
[inserirEmpresas] => Conexao com banco de dados: COLETADEDADOS efetuada
[inserirEmpresaBancoDados] => Iniciado
[inserirEmpresaBancoDados] => UUID do registro: c45ef247-3524-42ec-a0af-a2b64c91afac
[inserirEmpresaBancoDados] => Registro inserido: TONY VIDROS
[inserirBancoDados] => Finalizado
[inserirEmpresaBancoDados] => Iniciado
[inserirEmpresaBancoDados] => UUID do registro: 85caab96-4b1d-4df6-bd64-17772ec84f6f
[inserirEmpresaBancoDados] => Registro inserido: CALHAS A.V.R
[inserirBancoDados] => Finalizado
[inserirEmpresaBancoDados] => Iniciado
[inserirEmpresaBancoDados] => UUID do registro: 1b0b4424-4131-4f5b-af03-1a8333bf2043
[inserirEmpresaBanc

tonado, divisórias: eucatex, pvc, bp plus.',
  'infoOculta': '',
  'endereco': {},
  'cep': '89203-307',
  'cidade': 'Joinville',
  'estado': 'santa catarina',
  'bairro': 'Anita Garibaldi',
  'uf': 'SC',
  'telefones': [{'ddd': '47',
    'telefone': '34540055',
    'telefoneCompleto': '(47) 3454-0055',
    'whatsapp': 'false',
    'telefone0800': 'false'},
   {'ddd': '47',
    'telefone': '34384722',
    'telefoneCompleto': '(47) 3438-4722',
    'whatsapp': 'false',
    'telefone0800': 'false'},
   {'ddd': '47',
    'telefone': '991878476',
    'telefoneCompleto': '(47) 99187-8476',
    'whatsapp': 'true',
    'telefone0800': 'false'}],
  'telefonesCompletos': '(47) 3454-0055, (47) 3438-4722, (47) 99187-8476',
  'lat': None,
  'lng': None,
  'tipoLog': '',
  'logradouro': 'Rua Ottokar Doerffel',
  'numero': '1095',
  'qtdEnderecos': 1,
  'enderecos': [' Rua Ottokar Doerffel, 1095, Anita Garibaldi, Joinville - SC'],
  'end_anunciante': [{'telefone_anun': [{'ddd': '47',
      'telefone'

In [3]:
def getGuiaFacilCategories():
    print('\nGetGuiaFacilCategories started...\n')
    site = 'https://www.guiafacil.com/'
    print('Opening {}...'.format(site))
    html = urlopen(site)
    print('Parsing content...')
    bs = BeautifulSoup(html.read(),'html.parser')
    print('Find all categories...')
    divQuickleSearch = bs.find('div', { 'id': 'busca_rapida'})
    categories = divQuickleSearch.findAll('a', href=re.compile('^(?!http)(.)*$'))
    formattedCategories = []
    for category in categories:
        formattedCategories.append(unidecode(category['title'].lower().replace(' ', '-')))
    print('Categories found {}'.format(formattedCategories))
    print('\nGetGuiaFacilCategories finished...\n')
    return formattedCategories

In [4]:
def getCategoryPage(url, city, state):
    page = 'https://www.guiafacil.com/titulo/{}/{}/{}/'.format(url, city, state)
    try:
        html = urlopen(page)
        bs = BeautifulSoup(html.read(),'html.parser')
        divResults = bs.find('div', { 'id': 'resultados'})
    except Exception:
        divResults = 'Page {} not found!'.format(page)
    return divResults

In [ ]:
categories = getGuiaFacilCategories()
for category in categories:
    print(getCategoryPage(category, 'joinville', 'santa-catarina'))


GetGuiaFacilCategories started...

Opening https://www.guiafacil.com/...
Parsing content...


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Find all categories...
Categories found ['pizzarias', 'restaurantes', 'lanches', 'churrascaria', 'concessionarias', 'oficinas', 'autopecas', 'motos', 'materiais-de-construcao', 'moveis-sob-medida', 'arquitetos', 'marmores-e-granitos', 'clinicas-de-estetica', 'farmacias', 'hospitais', 'dentistas', 'eletricistas', 'serralheiros', 'fretes', 'informatica', 'supermercados', 'pet-shops', 'papelarias', 'imobiliarias']

GetGuiaFacilCategories finished...

